![New Relic](./assets/new-relic-logo.png)

# New Relic Playground

## Imports

Python Imports

In [ ]:
import json
import os

from textwrap import dedent

Third-party libraries

In [ ]:
import dotenv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import requests

from requests import Session, request

## Useful definitions

In [ ]:
NULL_CURSOR = json.dumps(None)

In [ ]:
def build_query(query_string, query_params = None):
    if query_params is None:
        query_params = {}
        
    return dedent(query_string.strip()) % query_params

In [ ]:
def get_response_data(response, key_path = None, action = "actor"):
    data = response.json().get("data").get(action)

    if key_path is not None:
        for key in key_path.split(":"):
            if key.isdecimal() and isinstance(data, list):
                data = data[int(key)]
            else:
                data = data.get(key)

    return data

In [ ]:
class NewRelicGqlClient(Session):
    """Client for New Relic GraphQL API."""

    url: str = "https://api.newrelic.com/graphql"

    def __init__(self, new_relic_user_key = None):
        super().__init__()

        self.headers.update(
            {
                "Content-Type": "application/json",
                "API-Key": new_relic_user_key,
            }
        )

    def execute(self, query, variables = None, **kwargs):
        data = json.dumps(
            {
                "query": query,
                "variables": variables,
            },
        )
        return self.post(self.url, data=data, **kwargs)

## Client setup

Load credentials

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

new_relic_user_key = os.environ.get("NEW_RELIC_USER_KEY", None)

if new_relic_user_key is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

Client instantiation

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key)

Define test query

In [ ]:
query = build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

Executing query

In [ ]:
response = newrelic.execute(query)

Looking for response data

In [ ]:
print(
    json.dumps(
        get_response_data(
            response, 
            action="actor",
        ),
        indent=4,
    )
)

## Playground area

### List all available service levels

In [ ]:
query_string="""
{
    actor {
        entitySearch(query: "type IN ('SERVICE_LEVEL')") {
            count
            query
            results {
                nextCursor
                entities {
                    accountId
                    alertSeverity
                    domain
                    entityType
                    firstIndexedAt
                    guid
                    indexedAt
                    lastReportingChangeAt
                    name
                    permalink
                    reporting
                    type
                }
            }
        }
    }
}
"""

query_params={
    "cursor": NULL_CURSOR,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
service_level_count = get_response_data(response, key_path="entitySearch:count", action="actor")
service_level_count

In [ ]:
service_level_list = get_response_data(response, key_path="entitySearch:results:entities", action="actor")
len(service_level_list)

In [ ]:
service_level_list[0]

In [ ]:
next_cursor = get_response_data(response, key_path="entitySearch:results:nextCursor", action="actor")
next_cursor

### Get a single service level

In [ ]:
service_level_guid = service_level_list[0]["guid"]

In [ ]:
query_string="""
{
    actor {
        entity(guid: "%(service_level_guid)s") {
            accountId
            alertSeverity
            domain
            entityType
            firstIndexedAt
            guid
            indexedAt
            lastReportingChangeAt
            name
            permalink
            reporting
            type
        }
    }
}
"""

query_params={
    "service_level_guid": service_level_guid,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
serivce_level = get_response_data(response, key_path="entity", action="actor")
serivce_level

### Task: Create a service level

In [ ]:
query_string="""
"""

query_params={
    "account_id": ACCOUNT_ID,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
created_service_level = get_response_data(response, action="put your key here")
created_service_level